<a href="https://colab.research.google.com/github/OceanZhang1988/CS4641/blob/main/Project_1_Release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to https://aritter.github.io/CS-7650/
 
# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)

#Project \#1: Decision Trees

In this assignment you will implement a decision tree learning algorithm that uses information gain as the splitting criterea.

In this notebook, we provide you with some starter code to read in the data and evaluate the performance of your implementation. After completing the instructions below, please follow the instructions at the end to submit your notebook and other files to Gradescope.

Make sure to make a copy of this notebook, so your changes are saved.

#Download the dataset

First, let's download the experimental data:

In [2]:
!wget https://raw.githubusercontent.com/aritter/CS-4641/main/data/train.data
!wget https://raw.githubusercontent.com/aritter/CS-4641/main/data/dev.data
!wget https://raw.githubusercontent.com/aritter/CS-4641/main/data/test.data

--2021-09-05 07:06:49--  https://raw.githubusercontent.com/aritter/CS-4641/main/data/train.data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230000 (225K) [text/plain]
Saving to: ‘train.data’

train.data          100%[===================>] 224.61K  --.-KB/s    in 0.02s   

2021-09-05 07:06:49 (13.5 MB/s) - ‘train.data’ saved [230000/230000]

--2021-09-05 07:06:49--  https://raw.githubusercontent.com/aritter/CS-4641/main/data/dev.data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46000 (45K) [text/plain]
Saving to: ‘dev.data’


Here is a look at what the data looks like.  Also note that in Colab you can run Linux shell commands by prepending them with a `!` like so:

In [ ]:
!head train.data

e,x,s,g,f,n,f,w,b,p,t,e,f,s,w,w,p,w,o,e,k,s,g
p,x,f,y,f,f,f,c,b,h,e,b,k,k,n,n,p,w,o,l,h,v,p
p,f,s,e,f,y,f,c,n,b,t,?,s,s,w,w,p,w,o,e,w,v,l
e,f,f,w,f,n,f,w,b,p,t,e,f,f,w,w,p,w,o,e,k,s,g
p,f,y,y,f,f,f,c,b,g,e,b,k,k,p,n,p,w,o,l,h,v,g
e,x,s,g,f,n,f,w,b,p,t,e,f,f,w,w,p,w,o,e,n,a,g
p,b,s,p,t,n,f,c,b,r,e,b,s,s,w,w,p,w,t,p,r,v,g
e,x,y,y,t,l,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,m
e,x,y,r,f,n,f,c,n,w,e,?,s,f,w,w,p,w,o,f,h,y,d
e,f,f,e,t,n,f,c,b,p,t,b,s,s,g,g,p,w,o,p,k,v,d


Each row describes a mushroom. The first column indicates whether it is edible 'e' or poisonous 'p' (this is the label we will try to predict).  The remaining columns are various discrete features which may be helpful in determining whether the mushroom is poisonous.  In case you were curious, the following is a description of the meaning of each feature.  **Disclaimer: please do not use your model's learned decision tree to determine whether to eat a mushroom!**

1. cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=*s*
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
4. bruises?: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d


#Dataloader

The code below reads in the data and represents each mushroom as a list of `FeatureVal` named tuples.

In [3]:
from collections import namedtuple
import sys
import math

FeatureVal = namedtuple("FeatureVal", "feature, value")        

class MushroomData:
    def __init__(self, fileName):
        self.data = []
        self.features = set()
        for line in open(fileName):
            line = line.strip()            
            attributes = line.split(',')
            for i in range(1,len(attributes)):
                self.features.add(FeatureVal(i, attributes[i]))
            self.data.append(attributes)

Let's see what that looks like:

In [4]:
train = MushroomData('train.data')
dev = MushroomData('dev.data')
test = MushroomData('test.data')

print("list of possible feature values indexed by position:", sorted(train.features))
print("first training example:", train.data[0])

list of possible feature values indexed by position: [FeatureVal(feature=1, value='b'), FeatureVal(feature=1, value='c'), FeatureVal(feature=1, value='f'), FeatureVal(feature=1, value='k'), FeatureVal(feature=1, value='s'), FeatureVal(feature=1, value='x'), FeatureVal(feature=2, value='f'), FeatureVal(feature=2, value='g'), FeatureVal(feature=2, value='s'), FeatureVal(feature=2, value='y'), FeatureVal(feature=3, value='b'), FeatureVal(feature=3, value='c'), FeatureVal(feature=3, value='e'), FeatureVal(feature=3, value='g'), FeatureVal(feature=3, value='n'), FeatureVal(feature=3, value='p'), FeatureVal(feature=3, value='r'), FeatureVal(feature=3, value='u'), FeatureVal(feature=3, value='w'), FeatureVal(feature=3, value='y'), FeatureVal(feature=4, value='f'), FeatureVal(feature=4, value='t'), FeatureVal(feature=5, value='a'), FeatureVal(feature=5, value='c'), FeatureVal(feature=5, value='f'), FeatureVal(feature=5, value='l'), FeatureVal(feature=5, value='m'), FeatureVal(feature=5, value=

We can see that there are many multi-valued attributes in the dataset. To simplify the implementation and visualization process of decision trees, we want to **convert multi-valued attributes into multiple binary attributes** by checking whether a feature has a specific value (see the example below). In this way, the generated decision trees will be **binary** trees.

In [5]:
f = sorted(train.features)[0]

print("Here is an example of how to test whether a feature has a specific value")
print(train.data[100][f.feature] == f.value)
print(train.data[0][f.feature] == f.value)

Here is an example of how to test whether a feature has a specific value
False
False


# ID3 algorithm (15 points)

First, try running the provided starter code in the cell below.

The provided code simply “learns” a decision tree consisting of a root node with no children that always predicts “Yes”. The program prints out this tree (consisting of a single node with no children), and then prints its evaluated accuracy on the test set which should be around 0.5.

Your job is to implement the functions InformationGain() and ID3() to grow a decision tree from the data.

In [6]:
DtNode = namedtuple("DtNode", "fVal, nPosNeg, gain, left, right")

POS_CLASS = 'e'
NEG_CLASS = 'p'
ALPHA = 1 # laplace smoothing

def InformationGain(data, f):
    #TODO: Calculate the information gain given a attribute/feature. (5 points).
    """
    Args:
        data:
            Input examples.
        f:
            A selected feature.

    Returns:
        :A (info. gain value, #pos examples, #neg examples) tuple
    """
    info_gain, npos, nneg = 0, 0, 0
    return (info_gain, npos, nneg)

def Classify(tree, instance):
    if tree.left == None and tree.right == None:
        return tree.nPosNeg[0] > tree.nPosNeg[1]
    elif instance[tree.fVal.feature] == tree.fVal.value:
        return Classify(tree.left, instance)
    else:
        return Classify(tree.right, instance)

def Accuracy(tree, data):
    nCorrect = 0
    for d in data:
        if Classify(tree, d) == (d[0] == POS_CLASS):
            nCorrect += 1
    
    print(f"The accuracy is: {float(nCorrect) / len(data)}")

def PrintTree(node, prefix=''):
    if prefix == '':
        print('root:')
    print("%s>next_split=%s\t edible/poisonous=%s\t info_gain=%s" % (prefix, node.fVal, node.nPosNeg, node.gain))
    if node.left != None:
        print(f"{prefix}{node.fVal}==True:")
        PrintTree(node.left, prefix + '-')
    if node.right != None:
        print(f"{prefix}{node.fVal}==False:")
        PrintTree(node.right, prefix + '-')    
    print()    

def SavePredictions(tree, data, outFile):
    fOut = open(outFile, 'w')
    for d in data:
        if Classify(tree, d):
            fOut.write(f"{POS_CLASS}\n")
        else:
            fOut.write(f"{NEG_CLASS}\n")
        
def ID3(data, features, MIN_GAIN=0.1):
    #TODO: Implement ID3 algorithm which outputs **binary** decision trees (10 points).
    #Hint: Make sure you use the converted binary features following the example above.
    """
    Args:
        data:
            Input examples.
        features:
            Input features sets.
        MIN_GAIN:
            A threhold for informaiton gain to make further splits.

    Returns:
        :A built decision tree (using DtNode defined above)
    """
    return DtNode(None, (0, 0), 0, None, None)

train = MushroomData('train.data')
dev = MushroomData('dev.data')
test = MushroomData('test.data')

dTree = ID3(train.data, train.features, MIN_GAIN=0.01)
    
PrintTree(dTree)

Accuracy(dTree, dev.data)

SavePredictions(dTree, test.data, 'test_pred.txt')


root:
>next_split=None	 edible/poisonous=(0, 0)	 info_gain=0

The accuracy is: 0.458


## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* Project_1_Release.ipynb
* test_pred.txt

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.

